In [1]:
# Importem llibreries
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
import pandas as pd
import colorsys
import graphviz

from pandas.plotting import scatter_matrix
from matplotlib.colors import ListedColormap

from sklearn import cluster
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import datasets, neighbors, tree, svm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree
from sklearn.metrics import classification_report, accuracy_score, plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import export_graphviz
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import silhouette_samples, silhouette_score
from statsmodels.graphics.mosaicplot import mosaic
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
import json
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from matplotlib import pyplot
import pickle
from joblib import dump, load
from mpl_toolkits.mplot3d.axes3d import Axes3D
from plotly.offline import plot
import plotly.graph_objs as go
import plotly
from sqlalchemy import create_engine 
from pandas import json_normalize
%matplotlib inline
plotly.offline.init_notebook_mode()

ModuleNotFoundError: No module named 'graphviz'

In [ ]:
# DB Constants
POSTGRES_ADDRESS = '45.76.81.71' 
POSTGRES_PORT = '5432' 
POSTGRES_USERNAME = 'airflow'
POSTGRES_PASSWORD = 'airflow' 
POSTGRES_DBNAME = 'dominis'

# Stablish connection
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT, dbname=POSTGRES_DBNAME)) 
conn = create_engine(postgres_str)

In [ ]:
def concat_frames(frames_list):
    return pd.concat(frames_list, axis=1)

def add_value_labels(ax, spacing=5):
    """
    Add labels to the end of each bar in a chart.
    This function was created by justfortherecs and improved by the
    author of this file.

    Source: https://stackoverflow.com/questions/28931224/adding-value-labels-on-a-matplotlib-bar-chart
    Arguments:
        ax (matplotlib.axes.Axes): The matplotlib object containing the axes
            of the plot to annotate.
        spacing (int): The distance between the labels and the bars.
    """

    # For each bar: Place a label
    for rect in ax.patches:
        # Get X and Y placement of label from rect.
        y_value = rect.get_height()
        x_value = rect.get_x() + rect.get_width() / 2

        # Number of points between bar and label. Change to your liking.
        space = spacing
        # Vertical alignment for positive values
        va = 'bottom'

        # If value of bar is negative: Place label below bar
        if y_value < 0:
            # Invert space to place label below
            space *= -1
            # Vertically align label at top
            va = 'top'

        # Use Y value as label and format number with one decimal place
        if isinstance(y_value, float):
            #try:
            #    label = int(y_value)
            #except ValueError:
            label = "{:.2f}".format(y_value)
        else:
            label = y_value

        # Create annotation
        ax.annotate(
            label,                      # Use `label` as label
            (x_value, y_value),         # Place label at end of the bar
            xytext=(0, space),          # Vertically shift label by `space`--
            textcoords="offset points", # Interpret `xytext` as offset in points
            ha='center',                # Horizontally center label
            va=va)                      # Vertically align label differently for
                                        # positive and negative values.

    return ax

def hist_groupby(df, var, category, alpha=0.6, ax=None):
    """
    Method that creates a histogram divided by 
    a class variable (category)
    """
    # Group by the class variable "category" and select the desired column
    groups = df.groupby(category)[var]
    
    if ax is None:
        # Create the plot
        fig, ax = plt.subplots()

    # For each group found for the variable "category" create a histogram and add it to the plot
    for k, v in groups:
        sns.distplot(v, ax=ax, label=k)
        #v.dist(label=k, alpha=alpha, ax=ax)

    # Activate the legend
    ax.legend()
    
    # Set the title and axis
    ax.set_title("{0} by {1}".format(var, category))
    ax.set_xlabel(var)
    ax.set_ylabel("Frequency")
        
    return ax

class EstimatorSelectionHelper:

    def __init__(self, models, params, scoring=['accuracy'], cv=5):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError("Some estimators are missing parameters: %s" % missing_params)
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}
        self.scoring = scoring
        self.cv = cv

    def fit(self, X, y, n_jobs=3, verbose=1, refit=False):
        for key in self.keys:
            print("Running GridSearchCV for %s." % key)
            model = self.models[key]
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=self.cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=self.scoring, refit=refit,
                              return_train_score=True)
            gs.fit(X,y)
            self.grid_searches[key] = gs

    def score_summary(self):
        def row(key, scores, params):
            d = {}
            d['estimator'] = key
            for score, values in scores.items():
                d['min_score_{}'.format(score)] = min(values)
                d['max_score_{}'.format(score)] = max(values)
                d['mean_score_{}'.format(score)] = np.mean(values)
                d['std_score_{}'.format(score)] = np.std(values)

            return pd.Series({**params, **d})

        rows = []
        for k in self.grid_searches:
            params = self.grid_searches[k].cv_results_['params']
            scores_list = {}
            for p_i, val in enumerate(params, start=0):
                #print("PARAMS {} at index {}".format(str(val), p_i))
                scores_list[str(val)] = {}
                for score in self.scoring:
                    #print("SCORE: {}".format(score))
                    scores_list[str(val)][score] = []
                    for i in range(0, self.cv):
                        key = "split{}_test_{}".format(i, score)
                        #print("KEY {}".format(key))
                        r = self.grid_searches[k].cv_results_[key]
                        #print("VAL {} at index {}".format(r, p_i))
                        scores_list[str(val)][score].append(r[p_i])

            for param in params:
                rows.append((row(k, scores_list[str(param)], param)))
        df = pd.DataFrame(rows)

        score_cols = []
        for score in self.scoring:
            score_cols += ['min_score_{}'.format(score), 'mean_score_{}'.format(score), 'max_score_{}'.format(score), 'std_score_{}'.format(score)]
        columns = ['estimator'] + score_cols
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns]

    @staticmethod
    def search_plot(df, score, x, y, ax=None):
        # TODO:  An automatic search given the whole score_summary should be implemented.
        df = df.pivot(index=y, columns=x, values=score)
        return sns.heatmap(df, ax=ax)


class PCAw(PCA):

    def __init__(self, *args, **kwargs):
        PCA.__init__(self, *args, **kwargs)

    def variance_plot(self, ax=None):
        """
        Method that given a PCA instance
        returns a plot with the relative
        and accumulative variance.
        """
        pc_variance_df = pd.DataFrame({
            'accumulative_variance': np.cumsum(self.explained_variance_ratio_),
            'relative_variance': self.explained_variance_ratio_
            })

        plot = pc_variance_df.plot(kind='bar', ax=ax)
        plot.axhline(y=0.95, color='r', linestyle='--')
        plot.set_title("PCA Explained variance")
        plot.set_xlabel("Principal Components")
        plot.set_ylabel("Variance")

        return plot

    def plot_contribution(self, index, columns, ax=None):
        eigenvalues=self.components_
        pc=abs(eigenvalues[index,:])
        contributions = pd.DataFrame({'contribution': pc}, index=columns)
        ax = contributions.sort_values(by='contribution', ascending=False).plot(kind='bar', title="Contribution of variables to DIM {}".format(index), ax=ax)
        ax.axhline(y=0.1, color='r', linestyle='--')
        return ax

def plot_silhouette_method(df, k_min=2, k_max=10, ax=None):
    silhouette = []
    for k in range(k_min, k_max):
        kmeans = cluster.KMeans(n_clusters=k)
        clusters = kmeans.fit_predict(df)
        silhouette.append(silhouette_score(df, clusters))
    if ax is not None:
        return ax.plot(range(k_min,k_max), silhouette, marker='o')
    else:
        return plt.plot(range(k_min,k_max), silhouette, marker='o')

def plot_elbow_method(df, k_min=1, k_max=10, ax=None):
    sse = []
    # Apliquem KMeans pel rang de k especificat
    for k in range(k_min, k_max):
        kmeans = cluster.KMeans(n_clusters=k)
        # Afegim les dades
        kmeans.fit(df)
        # Obtenim SSE
        sse.append(kmeans.inertia_)

    if ax is not None:
        return ax.plot(range(k_min, k_max), sse, marker='o')
    else:
        return plt.plot(range(k_min, k_max), sse, marker='o')

# Classificació d'empreses del domini .CAT

Al puntCAT, el 10 de gener de 2020 hi havien registrats un total de 108.445 dominis. L'objectiu d'aquest document és descriure el procediment seguit per arribar a determinar quins d'aquests dominis pertanyen a una empresa.

Aquest procediment consta de diverses jerarquies:

1. __Classificació segons la resposta__: Aquest primer nivell permet realitzar un filtratge ràpid de les URLs que no interessen analitzar. Per a fer-ho es realitza una petició GET al domini amb el mòdul de python `requests`. A partir d'aquí es considera:

    * El servidor respon? Si no ho fa el domini queda descartat.
    * El codi de resposta retornat és inferior a 400? Si no ho fa el domini queda descartat.
    * El servidor redirecciona a un altre domini? Si ho fa el domini queda descartat ja que es vol realitzar únicament l'estudi dels dominis ".cat". Cal tenir en compte que si el domini redirigeix a un altre dins del ".cat" també es pot descartar perquè apareixerà en el llistat original.
    
2. __Classificació segons el contingut__: Una parts dels dominis que donguin resposta no resultaran interessants d'analitzar a priori perquè correspondran a pàgines web buides o _landing pages_ d'empreses de tercers que es dediquen a vendre els dominis. En aquest cas per accedir a la pàgina web es realitzarà utilitzant un navegador que suporti l'execució de Javascript (Google Chrome controlat amb Selenium). A partir d'aquí es considera:

    * El navegador carrega la pàgina web? Si no ho fa el domini queda descartat.
    * La pàgina web redirecciona a un altre domini utilitzant Javascript? Si ho fa el domini queda descartat ja que es vol realitzar únicament l'estudi dels dominis ".cat". Cal tenir en compte, com abans, que si el domini redirigeix a un altre dins del ".cat" també es pot descartar perquè apareixerà en el llistat original.
    * Obtenir l'estructura de la pàgina web: mida de la pàgina, nombre de paraules, nombre de enllaços externs i interns i el nombre de vegades que apareixen un seguit de tags HTML: image, div, script, frame, style, meta i link. Amb aquestes dades utilitzar un model supervisat entrenat a partir de la classificació manual de 519 dominis que determini si la pàgina web presenta o no contingut. Si el model determina que no hi ha contingut el domini queda descartat.
    
3. __Classificació segons empresa__: En aquest punt es disposa del llistat de dominis que presenten una pàgina web dins del ".cat" amb contingut. L'últim pas consisteix en classificar aquests dominis segons si pertanyen o no a una empresa. A partir d'aquí es considera:
    * Obtenir diverses dades addicionals d'aquest domini:
        * Buscar enllaços en la *landing page* referents a avís legal, galetes i privacitat, ocurrènces de les paraules registre mercantil i domicili fiscal, xarxes socials (twitter, facebook, linkedin, instagram and youtube), NIFs, correus electònics i telèfons.
        * Per cada NIF trobat a la pàgina web buscar l'empresa associada amb l'*scraping* de [eInforma](https://www.einforma.com).
        * Detectar les tecnologies utilitzades per cada pàgina web utilitzant [Wappalyzer](https://www.wappalyzer.com).
        
     A partir de totes aquests dades i la classificació manual de 370 pàgines web crear un model supervisat que ens retorni la probabilitat de cada pàgina de pertànyer a una empresa.

Tot aquest procés de captura de les dades ha estat realitzat en la totalitat dues vegades. Una primera des d'un Mac Pro connectat a la xarxa de la URV i una segona vegada amb un servidor remot proporcionat per la Fundació puntCAT mitjançant un script que es pot trobar en el següent [repositori](https://github.com/REPUBLIQUEM-CAT/domain_classifier).

## Neteja de les dades

Abans de començar amb la classificació és necessari obtenir les dades emmagatzemades a la base de dades relacional i netejar-les per així poder treballar còmodament amb elles.

Primer de tot es realitza una _query_ que ens retornarà per cada domini tota la informació necessaria.

In [ ]:
query = """
SELECT dc.domain, dc.codi_resposta, dc.redirecciona_a_req, dc.mida, dc.paraules, dc.internal_link, dc.external_link, dc.images, dc.div, dc.script, dc.frame, dc.style, dc.meta, dc.link, dc.contingut, dc.resposta_sel, dc.redirecciona_a_sel, dc.avis_legal, dc.privacitat, dc.cookies, dc.contacte, dc.registre_mercantil, dc.domicili_fiscal, dc.twitter, dc.facebook, dc.linkedin, dc.instagram, dc.youtube, dc.es_empresa, dc.builtwith, email_found.email, phone_found.phone, nif_found.nif, nif_found.nif_verificat, domini_puntcat.empresa_dirigit, domini_puntcat.es_empresa_manual, dc.lighthouse
FROM domain_classification as dc
LEFT JOIN (
	SELECT search, domini, COUNT(*) as email FROM domain_email GROUP BY search, domini
	) as email_found
ON dc.search = email_found.search AND dc.domain = email_found.domini
LEFT JOIN (
	SELECT search, domini, COUNT(*) as phone FROM domain_phone GROUP BY search, domini
	) as phone_found
ON dc.search = phone_found.search AND dc.domain = phone_found.domini
LEFT JOIN (
SELECT COUNT(*) AS nif, SUM(CASE WHEN denominacio IS NULL THEN 0 ELSE 1 END) AS nif_verificat, domini, domain_nif.search
	FROM domain_nif
	LEFT JOIN empresa ON domain_nif.nif = empresa.nif
	WHERE domain_nif.search IN (SELECT search FROM (SELECT MAX(search) as search, domain FROM domain_classification GROUP BY domain) AS to_select)
	GROUP BY domini, domain_nif.search
) as nif_found
ON dc.search = nif_found.search AND dc.domain = nif_found.domini
LEFT JOIN domini_puntcat ON domini_puntcat.domini = dc.domain
WHERE dc.search IN (SELECT search FROM (SELECT MAX(search) as search, domain FROM domain_classification GROUP BY domain) AS to_select)
"""
# AND domini_puntcat.es_empresa_manual IS NOT NULL AND dc.response_url IS NOT NULL AND dc.codi_resposta = 200 AND dc.redirecciona_a_req IS NULL AND dc.contingut IS TRUE;
# Fetch query
contingut = pd.read_sql_query(query, conn)


Es comprova que les dades s'hagin carregat correctament.

In [ ]:
contingut.head()

El nombre de files i columnes obtingudes de la _query_ són respectivament:

In [ ]:
contingut.shape

Seguidament es llisten el nom de totes les columnes i es procedeix amb la neteja de les dades.

In [ ]:
contingut.dtypes

* __Columnes d'empresa__: La columna booleana corresponent a la variable classe `es_empresa_manual` serà separada del conjunt d'atributs. Addicionalment també ho serà la columna `empresa_dirigit` que són aquells dominis classificats manualment de manera dirigida. Finalment la columna que conté la probabilitat de pertànyer a una empresa `es_empresa` també serà separada.

In [ ]:
es_empresa = contingut.loc[:, ["es_empresa_manual"]]
es_empresa = es_empresa.applymap(lambda x: 1 if x is True else x)
es_empresa = es_empresa.applymap(lambda x: 0 if x is False else x)

empresa_dirigit = contingut.loc[:, ["empresa_dirigit"]]
empresa_dirigit = empresa_dirigit.applymap(lambda x: 1 if x is True else 0)

prob_empresa = contingut.loc[:, ["es_empresa"]]

* Les columnes de valors flotants `email`, `phone`, `nif`, `nif_verificat` seran convertides en binàries en funció de si tenen o no dades.

In [ ]:
epnn = contingut.loc[:, ["email", "phone", "nif", "nif_verificat"]]
epnn = epnn.fillna(0)
epnn = epnn.applymap(lambda x: 1 if x>0 else 0)

* Les tecnologies de la pàgina web presents a la columna `builtwith` seran separades per a crear un seguit de columnes binàries a partir d'aquesta.

In [ ]:
for index, row in contingut.iterrows():
    applications_list = {}
    data = row["builtwith"]
    if data is not None and "applications" in data:
            for application in data["applications"]:
                for category_number, category_name in application["categories"].items():
                    if category_name not in applications_list:
                        applications_list[category_name] = []
                    applications_list[category_name].append(application["name"])
    contingut.at[index, "builtwith"] = applications_list

In [ ]:
group_tecnologies_cols = []
group_tecnologies = contingut[["builtwith"]].copy()

for index, row in group_tecnologies.iterrows():
    keys = list(row["builtwith"].keys())
    for key in keys:
        if key not in group_tecnologies_cols:
            group_tecnologies_cols.append(key)
    
for column in group_tecnologies_cols:
    group_tecnologies[column] = 0

for index, row in group_tecnologies.iterrows():
    keys = list(row["builtwith"].keys())
    for key in keys:
        group_tecnologies.loc[index, key] = 1
    
group_tecnologies = group_tecnologies.drop(columns=["builtwith"])

* Les columnes referents a l'estructura de la pàgina web seran normalitzades entre 0 i 1.

In [ ]:
estructura_cols = ["mida", "paraules", "internal_link", "external_link", "images", "div", "script", "style", "meta", "link"]
estructura = contingut[estructura_cols]
estructura = MinMaxScaler().fit_transform(estructura)
estructura = pd.DataFrame(estructura, columns=estructura_cols)

* Les columnes referents al contingut de la pàgina web (avís legal, cookies, xarxes socials, etc) seran convertides també a binaries.

In [ ]:
contingut_object_cols = ["avis_legal", "privacitat", "cookies", "contacte", "twitter", "facebook", "instagram", "linkedin", "youtube"]
contingut_object = contingut[contingut_object_cols]
contingut_object = contingut_object.applymap(lambda x: 1 if x is not None else 0)


contingut_bool_cols = ['registre_mercantil', 'domicili_fiscal']
contingut_bool = contingut.loc[:, contingut_bool_cols]
booleanDictionary = {True: 1, False: 0}

for column in contingut_bool_cols:
    contingut_bool[column] = contingut[column].map(booleanDictionary)
#contingut_bool = contingut[contingut_bool_cols]
#contingut_bool = contingut_bool.applymap(lambda x: 1 if True else 0)

* Els diversos paràmetres de les auditories que conté la columna `lightouse` seran separats:

In [ ]:
def lighthouse_score(row):
    data = row["lighthouse"]
    if data is None:
        return None, None, None, None, None
    pwa = data["categories"]["pwa"]["score"]
    seo = data["categories"]["seo"]["score"]
    best_practices = data["categories"]["best-practices"]["score"]
    accessibility = data["categories"]["accessibility"]["score"]
    performance = data["categories"]["performance"]["score"]
    
    return pwa, seo, best_practices, accessibility, performance

In [ ]:
scores = contingut.apply(lighthouse_score, axis=1, result_type="expand")
score_columns = ["pwa", "seo", "best_practices", "accessibility", "performance"]
scores.columns = score_columns

In [ ]:
rows = []
for i, row in contingut.iterrows():
    audits = {"domain": row["domain"]}
    if row["lighthouse"] is not None:
        for key, value in row["lighthouse"]["audits"].items():
            audits[key] = value["score"]
        rows.append(audits)

audits = json_normalize(rows)
audits_cols = audits.columns

* Altres columnes que no necessiten transformació.

In [ ]:
altres_cols = ["domain", "codi_resposta", "redirecciona_a_req", "contingut", "resposta_sel", "redirecciona_a_sel"]
altres = contingut[altres_cols]

Finalment es junten totes les columnes.

In [ ]:
clean = concat_frames([es_empresa, empresa_dirigit, prob_empresa, epnn, estructura, contingut_object, contingut_bool, altres, group_tecnologies, scores, audits])  

In [ ]:
clean.head()

In [ ]:
clean.shape

Addicionalment s'exporta la taula neta a un fitxer CSV.

In [ ]:
clean.to_csv("empreses_lightohuse_only_manually_classified.csv", index=False)

## Classificació segons la resposta

In [ ]:
clean = pd.read_csv("empreses_some_builtwith.csv")

Aquest primer nivell permet realitzar un filtratge ràpid de les URLs que no interessen analitzar. Per a fer-ho es realitza una petició GET al domini amb el mòdul de python `requests`.

A partir d'aquí es determina si el servidor respon a la petició i si ho fa amb quin codi de resposta. En el següent gràfic es mostren els servidors que no han respost amb el `codi_resposta` -1.

In [ ]:
clean.codi_resposta = clean.codi_resposta.fillna(-1)
ax = clean.groupby("codi_resposta").size().plot(kind='bar', figsize=(15,7), title="Codis resposta dels dominis")
add_value_labels(ax)

De tots aquests dominis seguiran en el llistat de posibles empreses aquells que hagin retornat un codi de resposta més petit de 400 i major o igual a 200. Els [codis de resposta](https://developer.mozilla.org/es/docs/Web/HTTP/Status) 3XX corresponen a redireccions i els 2XX a èxit mentre que els 4XX a un error del client i els 5XX a un error del serividor. Cal destacar el codi resposta 999 que és retornat usualment per Linkedin al realitzar peticions automatitzades a la seva pàgina web.

Agregant aquestes condicions esmentades en el gràfic següent es mostren els dominis que segueixen en consideració. Addicionalment es proporciona una comparació entre la última execució (Juliol 2020)  i la primera execució (Febrer 2020) on es pot veure la pèrdua d'una mica més de 6.000 dominis. Aquest fet podria ser degut a la situació actual del COVID-19 i la cancel·lació dels serveis que mantenen el domini operatiu.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,7))
plt.suptitle("Codis resposta OK vs KO")

resposta_ok = clean[(clean.codi_resposta < 400) & (clean.codi_resposta > 0) ]
resposta_ko = clean[(clean.codi_resposta > 400) | (clean.codi_resposta < 0) ]
#classificacio_resposta = pd.DataFrame({"resposta": ["resposta_ok", "resposta_ko"], "count": [resposta_ok, resposta_ko]})
classificacio_resposta = pd.DataFrame(data=[resposta_ok.shape[0], resposta_ko.shape[0]], index=["resposta_ok", "resposta_ko"])
classificacio_resposta.plot.bar(rot=0, title="Execució juliol 2020", ax=ax1)
add_value_labels(ax1)


classificacio_old = pd.DataFrame(data=[84037, 24407], index=["resposta_ok", "resposta_ko"])
classificacio_old.plot.bar(rot=0, title="Execució febrer 2020", ax=ax2)
add_value_labels(ax2)


EL següent filtratge de dominis ve determinat per les redireccions que aquests presenten. Com que únicament es volen estudiar els dominis ".cat" tots aquells que presentin una redirecció seran automàticament descartats. Cal esmentar que aquells que presentin una redirecció a un altre domini ".cat" també seran descartats perquè ja estaran implícits en el llistat original.

In [ ]:
sense_redirecció = resposta_ok[resposta_ok.redirecciona_a_req.isna()]
amb_redirecció = resposta_ok[resposta_ok.redirecciona_a_req.isna() == False]
classificacio_resposta = pd.DataFrame(data=[sense_redirecció.shape[0], amb_redirecció.shape[0]], index=["Sense redirecció", "Amb redirecció"])
ax = classificacio_resposta.plot.bar(rot=0, figsize=(15,7), title="Dominis sense redirecció VS amb redirecció")
add_value_labels(ax)

Finalment es mostren els 10 dominis amb més redireccions.

In [ ]:
resposta_ok.groupby("redirecciona_a_req").size().to_frame().sort_values(by=0, ascending=False).head(10)

## Classificació segons el contingut

El següent punt en l'anàlisi dels dominis serà determinar si aquests presenten suficient informació per a ser analitzats. Alguns d'aquests dominis seran propietat de tercers com Nominalia, XXXXXX o XXXX entre d'altres que es dediquen a la venda de dominis o bé seran únicament _landing pages_ en blanc o amb l'índex de fitxers presents en el servidor. Com és evident tots aquests dominis es descartaran perquè no presenten prou contingut rellevant per a ser analitzats. L'objectiu final d'aquest apartat serà la creació d'un model supervisat que determini si tenen o no contingut, per a fer-ho es capturaran diversos aspectes de la pàgina web com la mida de la pàgina, nombre de paraules, nombre de enllaços externs i interns i el nombre de vegades que apareixen un seguit de tags HTML com image, div, script, frame, style, meta o link.

El fet és que per arribar a determinar de manera correcta aquests aspectes de la pàgina, aquesta ha de ser carregada per complet el que significa l'ús de Javascript. Per a procedir doncs s'accedirà a la pàgina web utilitzant un navegador que suporti l'execució de Javascript, que en aquest cas serà Google Chrome automatitzat amb [Selenium](https://selenium-python.readthedocs.io) mitjançant Python.


Abans de procedir amb la creació del model, l'accés als dominis mitjançant Google Chrome pot suposar que alguns dominis no responguin o bé que redirigeixin a un altre mitjançant Javascript. Per aixó es torna a realitzar el mateix filtratge que en el primer apartat, i en aquest cas el primer nivell queda determinat per si el navegador és capaç de carregar la pàgina web.

In [ ]:
resposta_ok_sel = sense_redirecció[sense_redirecció.resposta_sel == True]
resposta_ko_sel =  sense_redirecció[sense_redirecció.resposta_sel == False]
classificacio_resposta = pd.DataFrame(data=[resposta_ok_sel.shape[0], resposta_ko_sel.shape[0]], index=["Resposta OK Selenium", "Resposta KO Selenium"])
ax = classificacio_resposta.plot.bar(rot=0, figsize=(15,7), title="Codis resposta Selenium OK vs KO")
add_value_labels(ax)

El següent punt és la redirecció a un altre domini.

In [ ]:
sense_redirecció_sel = resposta_ok_sel[resposta_ok_sel.redirecciona_a_sel.isna()]
amb_redirecció_sel = resposta_ok_sel[resposta_ok_sel.redirecciona_a_sel.isna() == False]
classificacio_resposta = pd.DataFrame(data=[sense_redirecció_sel.shape[0], amb_redirecció_sel.shape[0]], index=["sense_redirecció_sel", "amb_redirecció_sel"])
ax = classificacio_resposta.plot.bar(rot=0, figsize=(15,7), title="Sense redirecció VS amb redirecció")
add_value_labels(ax)

Igual que en l'apartat anterior es mostren els 10 dominis més populars per a les redireccions.

In [ ]:
resposta_ok_sel.groupby("redirecciona_a_sel").size().to_frame().sort_values(by=0, ascending=False).head(10)

En aquest punt gràcies a la utilització d'un navegador automatitzat s'han pogut capturar els següents aspectes de la pàgina web:
* Mida.
* Nombre de paraules.
* Nombre d'enllaços interns.
* Nombre d'enllaços externs
* Nombre d'aparicions de cadascuna de les següents _tags_ HTML: image, div, script, frame, style, meta i link.

Després de la classificació manual de 519 dominis amb/sense contingut es decideix aplicar el model `RandomForest`. La mostra va ser dividida 70/30 i es va etrar el model amb la generació de 500 arbres de decisió. Aquest model va ser validat amb el conjunt de test obtenint les mètriques que es mostren a continuació.


In [ ]:
metriques = pd.DataFrame(data=[92.9, 90.6, 5.4, 8, 86.9, 94.6], index=["accuracy", "f1_score", "fnr", "fpr", "precision", "recall"])
ax = metriques.plot.bar(rot=0, title="Mètriques RandomForest (%)", figsize=(5, 6))
add_value_labels(ax)



En aplicar el model a la totalitat de les dades es va obtenir la següent classificació:

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,7))
plt.suptitle("Amb contingut Selenium VS sense contingut Selenium")

amb_contingut = sense_redirecció_sel[sense_redirecció_sel.contingut == True]
sense_contingut = sense_redirecció_sel[sense_redirecció_sel.contingut == False]
classificacio_resposta = pd.DataFrame(data=[amb_contingut.shape[0], sense_contingut.shape[0]], index=["Amb contingut Selenium", "Sense contingut Selenium"])
classificacio_resposta.plot.bar(rot=0, title="Execució juliol 2020", ax=ax1)
add_value_labels(ax1)

classificacio_old = pd.DataFrame(data=[34701, 19965], index=["Amb contingut Selenium", "Sense contingut Selenium"])
classificacio_old.plot.bar(rot=0, title="Execució febrer 2020", ax=ax2)
add_value_labels(ax2)

Com que el nombre de dominis sense contingut va ser bastant elevat i suposava un 36.5% de la totalitat dels dominis restants (54666) es va buscar una manera alternativa per determinar si un domini té o no contingut i contrastar-ho amb els resultats del model.
El que es va decidir estudiar va ser la similitud entre els dominis, en el cas que hi hagin molts dominis que presentin la mateixa estructura interna (tags HTML) es podria dir que són el mateix domini. Un clar exemple són totes aquelles pàgines que retornen un HTML buit o la mateixa estructura de la pàgina per defecte d'un domini que està en venda.

Per dir que dos dominis "eren el mateix" es va fixar el límit en més de tres dominis, és a dir, si 4 o més dominis s'agrupaven en la mateixa estructura es consideren dominis repetits, en cas contrari no.

En comparar els resultats es va observar que:

* En ambdós casos els dominis classificats sense contingut van ser 16.975.
* 1.687 dominis van ser considerats sense contingut per la similitud entre dominis i amb contingut pel Random Forest.
* 2.990 dominis van ser considerats amb contingut per la similitud entre dominis i sense contingut pel Random Forest.

Veient doncs com amdós mètodes donaven resultats molt similars es va decidir donar per bo el model del Random Forest.

## Classificació segons empresa

En aquest punt es disposa del llistat de dominis que presenten una pàgina web dins del ".cat" amb contingut. L'últim pas consisteix en classificar aquests dominis segons si pertanyen o no a una empresa. A partir d'aquí es considera:
* Obtenir diverses dades addicionals d'aquest domini:
    * Buscar enllaços en la *landing page* referents a avís legal, galetes i privacitat, ocurrènces de les paraules registre mercantil i domicili fiscal, xarxes socials (twitter, facebook, linkedin, instagram and youtube), NIFs, correus electònics i telèfons.
    * Per cada NIF trobat a la pàgina web buscar l'empresa associada amb l'*scraping* de [eInforma](https://www.einforma.com).
    * Detectar les tecnologies utilitzades per cada pàgina web utilitzant [Wappalyzer](https://www.wappalyzer.com).
        
  A partir de totes aquests dades i la classificació manual de 479 pàgines web crear un model supervisat que ens retorni la probabilitat de cada pàgina de pertànyer a una empresa.

In [ ]:
clean = resposta_ok_sel[resposta_ok_sel.es_empresa_manual.isna() == False]

In [ ]:
clean.shape

Per determinar si són o no empreses es disposen de tres grups de dades
* Les obtingudes a partir d'analitzar el contingut de la pàgina web. En aquest cas totes les variables són binàries (1 o 0) en funció de si s'ha detectat la condició o no.
    * Enllaç a avís legal, privacitat, galetes i contacte.
    * Aparició de les paraules registre mercantil i domicili fiscal.
    * Perfils de xarxes socials (Twitter, Facebook, Instagram, Youtube i Linkedin).
    * Correu electrònic i número de telèfon.
    * NIF i la seva corresponent verificació a [eInforma](https://www.einforma.com) (és a dir si hi ha una empresa registrada darrera del NIF).
* Les obtingudes anteriorment en l'anàlisi de l'estructura de la web (si tenen o no contingut). En aquest cas els valors es troben normalitzats entre 0 i 1.
    * Mida.
    * Nombre de paraules.
    * Nombre d'enllaços interns.
    * Nombre d'enllaços externs
    * Nombre d'aparicions de cadascuna de les següents _tags_ HTML: image, div, script, frame, style, meta i link.
* Les obtingudes després d'analitzar les tecnologies utilitzades per cada pàgina web com pot ser l'´ús d'una plataforma de e-commerce, Google Analytics, SEO o CMS entre molts d'altres. En aquest cas les variables són binàries (1 o 0) en funció de si s'ha detectat la condició o no.

Les columnes referents a cada grup es veuen a continuació.

In [ ]:
contingut_cols = ['registre_mercantil', 'domicili_fiscal', 'avis_legal', 'privacitat', 'cookies', 'contacte', 'twitter', 'facebook', 'linkedin', 'instagram', 'youtube', 'email', 'phone', 'nif', 'nif_verificat']
estructura_cols = ['mida', 'paraules', 'internal_link', 'external_link', 'images', 'div', 'script', 'style', 'meta', 'link']
group_tecnologies_cols = ['Analytics', 'Font scripts', 'Web frameworks', 'Web servers', 'Programming languages', 'Operating systems', 'Tag managers', 'JavaScript libraries', 'UI frameworks', 'CMS', 'Blogs', 'Widgets', 'Ecommerce', 'Captchas', 'Miscellaneous', 'Databases', 'Maps', 'Reverse proxies', 'SEO', 'PaaS', 'Mobile frameworks', 'Live chat', 'Video players', 'Editors', 'JavaScript graphics', 'JavaScript frameworks', 'Issue trackers', 'Hosting panels', 'CDN', 'Caching', 'Advertising', 'Page builders', 'Web server extensions', 'Static site generator', 'Comment systems', 'Payment processors', 'Marketing automation', 'Database managers', 'Photo galleries', 'Rich text editors', 'Webmail', 'LMS', 'Development', 'Message boards', 'Search engines', 'Network storage', 'DMS', 'Cryptominers', 'Wikis', 'Documentation tools']
no_nul_audits_cols = ['no-vulnerable-libraries', 'js-libraries', 'maskable-icon', 'splash-screen', 'installable-manifest', 'no-document-write', 'notification-on-start', 'offline-start-url', 'password-inputs-can-be-pasted-into', 'uses-http2', 'uses-passive-event-listeners', 'meta-description', 'http-status-code', 'font-display', 'geolocation-on-start', 'external-anchors-use-rel-noopener', 'dom-size', 'charset', 'html-has-lang', 'uses-long-cache-ttl', 'total-byte-weight', 'render-blocking-resources', 'unminified-css', 'unminified-javascript', 'unused-css-rules', 'aria-hidden-body', 'uses-webp-images', 'uses-optimized-images', 'uses-text-compression', 'uses-responsive-images', 'efficient-animated-content', 'appcache-manifest', 'doctype', 'uses-rel-preconnect', 'document-title', 'uses-rel-preload', 'mainthread-work-breakdown', 'viewport', 'speed-index', 'plugins', 'hreflang', 'works-offline', 'bootup-time', 'max-potential-fid', 'cumulative-layout-shift', 'errors-in-console', 'server-response-time', 'first-contentful-paint', 'largest-contentful-paint', 'tap-targets', 'is-crawlable', 'apple-touch-icon', 'themed-omnibox', 'content-width', 'image-aspect-ratio', 'image-size-responsive', 'is-on-https', 'link-text', 'deprecations', 'service-worker']
score_cols = ["pwa", "seo", "best_practices", "accessibility", "performance"]

Seguidament es procedeix a visualitzar cadascun dels grups de dades.

* __Contingut de la pàgina web__:

In [ ]:
fig, ax_temp = plt.subplots(4,4, figsize=(20, 20))
ax_list = []
for i in ax_temp:
    for j in i:
        ax_list.append(j)
        
for col, ax in zip(contingut_cols, ax_list):
    data = pd.concat([clean[[col]], clean[["es_empresa_manual"]]], axis=1)
    mosaic(data, [col, "es_empresa_manual"], ax=ax)
    ax.set_xlabel(xlabel=col)
    ax.set_ylabel(ylabel="Empresa")

Per cada columna es realitza un test d'independència de chi quadrat.

In [ ]:
p_value = chi2(clean[contingut_cols], clean.es_empresa_manual)[1]
pd.DataFrame({"cols": contingut_cols, "p_value": p_value}).sort_values(by='p_value').query("p_value < 0.05")

En aquest cas es pot obsevar com tots els p-valors són inferiors a 0.05 i per tant es descarta la hipòtesi nul·la i es pot dir que hi ha una relació entre ser una empresa i cadascuna de les columnes. Addicionalment amb els gràfics també es pot apreciar la importància de totes les variables en funció de si són o no empreses.

* __Estructura de la pàgina web__:

In [ ]:
fig, ax_temp = plt.subplots(5,2, figsize=(20, 30))
ax_list = []
for i in ax_temp:
    for j in i:
        ax_list.append(j)
        
for col, ax in zip(estructura_cols, ax_list):
    plot = hist_groupby(clean, col, "es_empresa_manual", ax=ax)

En aquest cas a simple vista no s'aprecia una gran diferència en la distribució de les variables. Tot i així les que visualment mostren més diferència entre grups són mida, paraules i images.

* __Tecnologies de la pàgina web__:

In [ ]:
fig, ax_temp = plt.subplots(13,4, figsize=(20, 50))
ax_list = []
for i in ax_temp:
    for j in i:
        ax_list.append(j)
        
for col, ax in zip(group_tecnologies_cols, ax_list):
    data = pd.concat([clean[[col]], clean.es_empresa_manual], axis=1)
    mosaic(data, [col, "es_empresa_manual"], ax=ax)
    ax.set_xlabel(xlabel=col)
    ax.set_ylabel(ylabel="Empresa")

* Audtories:

Per cada columna es realitza un test d'independència de chi quadrat.

In [ ]:
p_value = chi2(clean[group_tecnologies_cols], clean.es_empresa_manual)[1]
tecnologies_chi2 = pd.DataFrame({"cols": group_tecnologies_cols, "p_value": p_value})
tecnologies_chi2.sort_values(by='p_value').query("p_value < 0.05").to_latex("teconologies.text", index=False)

En aquest cas es pot obsevar com 10 dels p-valors són inferiors a 0.05 i per tant es descarta la hipòtesi nul·la i es pot dir que hi ha una relació entre ser una empresa i aquestes columnes.

In [ ]:
fig, ax_temp = plt.subplots(15,4, figsize=(20, 50))
ax_list = []
for i in ax_temp:
    for j in i:
        ax_list.append(j)
        
for col, ax in zip(no_nul_audits_cols, ax_list):
    data = pd.concat([clean[[col]], clean.es_empresa_manual], axis=1)
    mosaic(data, [col, "es_empresa_manual"], ax=ax)
    ax.set_xlabel(xlabel=col)
    ax.set_ylabel(ylabel="Empresa")

In [ ]:
p_value = chi2(clean[no_nul_audits_cols], clean.es_empresa_manual)[1]
audits_chi2 = pd.DataFrame({"cols": no_nul_audits_cols, "p_value": p_value})
audits_chi2.sort_values(by='p_value').query("p_value < 0.05")

In [ ]:
clean[no_nul_audits_cols + ["domain", "es_empresa_manual"]]




Una vegada visualitzades cadascuna de les columnes es procedirà a la creació del model. A priori s'estudiaran dos models supervisats `SVM` i `RandomForest`, i com afecta la inclusió o no d'algunes variables al model (columnes contingut, estructura i tecnologies).

El procediment per a l'estudi serà el següent:

* Seleccionar les columnes pertinents i dividir el conjunt de dades en entrenament i test 70/30.
* Utilitzar 5-fold cross-validation sobre el conjunt d'entrenament (70%) per realitzar el _tunning_ dels paràmetres i estimar el comportament de cada model (SVM i RandomForest). La literatura científica sol assignar de manera arbitrària el valor de 5 o 10 a k. En aquest cas al no tenir una mostra de dades molt gran es decideix optar pel valor 5.
* Validar el model amb el conjunt de test (validació) mai vist per aquest.

Per a SVM es provaran els tres kernels: linear, radial basis function (rbf) i sigmoid. Addicionalment els hiperparàmetres C i gamma.
* __C__: paràmetre de regularització. Com més gran és C més petit és el marge, i com més petit C més gran el marge.
* __gamma__: Defineix com de lluny arriba la influència d'una observació d'entrenament. Com més petit el valor més lluny arriba la influència i com més gran menys lluny.

In [ ]:
models_svm = {
    'SVC': SVC()
}

params_svm = {
    'SVC': [
        {'kernel': ['linear'], 'C': [0.01, 0.1, 1, 10, 50, 100]},
        {'kernel': ['rbf'], 'C': [0.01, 0.1, 1, 10, 50, 100], 'gamma': [0.001, 0.0001, 0.01, 0.1]},
        {'kernel': ['sigmoid'], 'C': [0.01, 0.1, 1, 10, 50, 100], 'gamma': [0.001, 0.0001, 0.01, 0.1]}
    ]
}

Per a Random Forest es testejaran els hiperparàmmetres nombre d'estimadors i màxima profunditat:
* __n_estimators__: Nombre d'arbres de classificació a crear.
* __max_depth__: Màxima profunditat dels arbres.

In [ ]:
models_rfc = {
    'RandomForestClassifier': RandomForestClassifier(),
}

params_rfc = {
    'RandomForestClassifier': { 
        'n_estimators': [16, 32, 64, 100, 200, 500], 
        'max_depth': [2,4,8, 10, 20, 50, 100] },
}

Per mantenir el conjunt d'entrenament i test igual al llarg del document es dividirà una vegada a l'inici la mostra de les dades.

In [ ]:
# Seleccionem totes les columnes possibles
all_cols = contingut_cols + tecnologies_chi2.query("p_value < 0.05").cols.values.tolist() + ["mida", "paraules", "images"]
# Dividim 70/30
X_train, X_test, y_train, y_test = train_test_split(clean, clean.es_empresa_manual, test_size=0.3)
# Retornem els noms a les columnes
X_train = pd.DataFrame(data=X_train, columns=clean.columns)
X_test = pd.DataFrame(data=X_test, columns=clean.columns)

### Models utilitzant les columnes de "contingut"


En aquest cas s'estudia únicament com es comporten les columnes "contingut", obtingudes de l'scraping genèric de la pàgina web.

#### SVM

Apliquem 5-fold cross-validation sobre el conjunt d'entrenament i visualitzem quins presenten el millor comportament. S'ordenen els resultats basats en la precisió mitjana obtinguda.

In [ ]:
helper1 = EstimatorSelectionHelper(models_svm, params_svm, scoring=["accuracy"])
helper1.fit(X_train[contingut_cols], y_train, n_jobs=2)

scores = helper1.score_summary()
scores.sort_values(by="mean_score_accuracy", ascending=False).head(10)

Visualitzem el comportament dels hiperparàmetres.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,5))
ax1.set(title="SVM rbf kernel")
helper1.search_plot(scores.query("estimator == 'SVC' and kernel == 'rbf'"), "mean_score_accuracy", "C", "gamma", ax=ax1)
ax2.set(title="SVM linear kernel")
helper1.search_plot(scores.query("estimator == 'SVC' and kernel == 'linear'"), "mean_score_accuracy", "C", "gamma", ax=ax2)
ax3.set(title="SVM sigmoid kernel")
helper1.search_plot(scores.query("estimator == 'SVC' and kernel == 'sigmoid'"), "mean_score_accuracy", "C", "gamma", ax=ax3)

Els dos kernels més afectats pels hiperparàmetres són rbf i sigmoid ambdos amb comportaments idèntics. Per a valors de gamma molt petits la precisió decreix i aquesta es contraresta amb valors alts de C. A mesura que s'incrementa gamma i es decrementa C la precisió mitjana es manté molt estable (80%-85%).

Finalment s'entrena el model amb el conjunt d'entrenament, i els paràmetres que han obtingut millor resultat. En aquest cas ens quedem amb el kernel sigmoid amb C=50 i gamma=0.001. En aquest cas la precisió mínima és de les més elevades i la precisió màximes també.

In [ ]:
svm_c = SVC(C=50, gamma=0.001, kernel='sigmoid', probability=True)
svm_c.fit(X_train[contingut_cols], y_train)
svm_c.score(X_test[contingut_cols], y_test)

Després de diverses execucions cal dir que SVM presenta en tot moments uns resultats bastant estables amb precisions d'entre el 83% i el 86%. A més, els hiperparàmetres escollits es repeteixen independentment del split realitzat d'entrenament i test.

Addicionalment s'estudiarà la distribució de la predicció de la probabilitat de ser una empresa i la matriu de confusió associada.

In [ ]:
predicted_c = pd.DataFrame(data={"predicted_empresa": svm_c.predict_proba(X_test[contingut_cols])[:, 1], "es_empresa": y_test})
ax = sns.FacetGrid(predicted_c, hue="es_empresa", height=5).map(sns.distplot, "predicted_empresa", bins=20, kde=False).add_legend()
ax.set(title="Distribució de la predicció de probabilitat de ser empresa")
ax.fig.axes[0].axvline(0.5, color="red", linestyle="--")

In [ ]:
plot_confusion_matrix(svm_c, X_test[contingut_cols], y_test)

Després d'examinar els falsos positius, falsos negatius i la distribució de la predicció d'empreses podem dir que:
* El model és capaç d'identificar correctament aquells dominis que __clarament__ són d'una empresa. És a dir, aquelles pàgines web ben construides amb la majoria de la informació rellevant que aquestes poden tenir (telèfon, correu elèctronic, nif, enllaços d'interès).
* Una gran quantitat dels falsos negatius són d'aquells dominis que pertanyen a una empresa però basant-nos en els paràmetres extrets, les dades són nules o pràcticament nules i per tant el model no pot determinar que es tracten d'empresa. Recordem que una gran quantitat dels verdaders negatius presenten una probabilitat d'entre el 0% i el 20% que pertanyen als dominis dels quals els paràmetres trobats són practicament nuls.
* Evidentment trobem una zona d'incertesa entre el 40% i el 60% on nosaltres personalment podrem posar la barrera de tall on creiem més convenient. És a dir, quedar-nos com a empreses aquelles observacions que tenen una probabilitat major del 60% decrementant així els falsos positius (a costa dels falsos negatius).
* La zona més preocupant es troba en aquelles observacions que no són empreses però l'algorisme determina amb una probabilitat d'entre el 80% que si que ho són. Aquest fet és degut a la alta importància que li assigna el model a les variables `email` i `phone` que de manera individual assignen una probabilitat del 38.6% i 52% respectivament i conjuntament la fan pujar al 78.61. Els 12 falsos positius del model es troben tots aquí.




#### Random Forest

Apliquem 5-fold cross-validation sobre el conjunt d'entrenament i visualitzem quins presenten el millor comportament. S'ordenen els resultats basats en la precisió mitjana obtinguda.

In [ ]:
helper1 = EstimatorSelectionHelper(models_rfc, params_rfc, scoring=["accuracy", "f1"])
helper1.fit(X_train[contingut_cols], y_train, n_jobs=2)

scores = helper1.score_summary()
scores.sort_values(by="mean_score_accuracy", ascending=False).head(10)

Visualitzem el comportament dels hiperparàmetres.

In [ ]:
helper1.search_plot(scores.query("estimator == 'RandomForestClassifier'"), "mean_score_accuracy", "n_estimators", "max_depth")

Entrenem el model amb els hiperparàmetres escollits.

In [ ]:
rfc_c = RandomForestClassifier(n_estimators=100, max_depth=8)
rfc_c.fit(X_train[contingut_cols], y_train)
rfc_c.score(X_test[contingut_cols], y_test)

En aquest cas el model Random Forest té un comportament molt similar al SVM creat anteriorment amb precisions igual d'entre el 83 i 86%.

Com abans, estudiem la distribució i la matriu de confusió.

In [ ]:
predicted_rfc_c = pd.DataFrame(data={"predicted_empresa": rfc_c.predict_proba(X_test[contingut_cols])[:, 1], "es_empresa": y_test})
ax = sns.FacetGrid(predicted_rfc_c, hue="es_empresa", height=5).map(sns.distplot, "predicted_empresa", bins=20, kde=False).add_legend()
ax.set(title="Distribució de la predicció de probabilitat de ser empresa")
ax.fig.axes[0].axvline(0.5, color="red", linestyle="--")

In [ ]:
plot_confusion_matrix(rfc_c, X_test[contingut_cols], y_test)

Com es pot observar en el gràfic de distribució i en la matriu de confusió el comportament és gairebé idèntic al SVM. Pel que fa als falsos positius els 12 dels 13 dominis classificats incorrectament coincideixen amb els del SVM.

Vist doncs el comportament gairebé idèntic dels models i el gran biaix que suposen les variables `phone` i `email` es seguirà amb les pautes incialmet creades: estudiar com varia el model amb la inclusió o no d'algunes variables. Com s'espera que el model no variï significativament es realitzaran els mateixos passos que els realitzats en aquest apartat i es comentaran els resultats en l'apartat [1.4.6 - Conclusions models](#conclusions_model).

### Models utilitzant contingut i estructura

En aquest cas s'estudia únicament com es comporten les columnes "contingut" i "estructra" més significatives.

In [ ]:
ce_cols = contingut_cols + ["mida", "paraules", "images"]

#### SVM

Apliquem 5-fold cross-validation sobre el conjunt d'entrenament i visualitzem quins presenten el millor comportament. S'ordenen els resultats basats en la precisió mitjana obtinguda.


In [ ]:
helper1 = EstimatorSelectionHelper(models_svm, params_svm, scoring=["accuracy"])
helper1.fit(X_train[ce_cols], y_train, n_jobs=2)

scores = helper1.score_summary()
scores.sort_values(by="mean_score_accuracy", ascending=False).head(10)

Visualitzem el comportament dels hiperparàmetres.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,5))
ax1.set(title="SVM amb kernel rbf")
helper1.search_plot(scores.query("estimator == 'SVC' and kernel == 'rbf'"), "mean_score_accuracy", "C", "gamma", ax=ax1)
ax1.set(title="SVM amb kernel linear")
helper1.search_plot(scores.query("estimator == 'SVC' and kernel == 'linear'"), "mean_score_accuracy", "C", "gamma", ax=ax2)
ax1.set(title="SVM amb kernel sigmoid")
helper1.search_plot(scores.query("estimator == 'SVC' and kernel == 'sigmoid'"), "mean_score_accuracy", "C", "gamma", ax=ax3)

Entrenem el model amb els hiperparàmetres escollits.


In [ ]:
svm_ce = SVC(C=50, gamma=0.01, kernel='sigmoid')
svm_ce.fit(X_train[ce_cols], y_train)
svm_ce.score(X_test[ce_cols], y_test)

#### Random Forest

Apliquem 5-fold cross-validation sobre el conjunt d'entrenament i visualitzem quins presenten el millor comportament. S'ordenen els resultats basats en la precisió mitjana obtinguda.


In [ ]:
helper1 = EstimatorSelectionHelper(models_rfc, params_rfc, scoring=["accuracy"])
helper1.fit(X_train[ce_cols], y_train, n_jobs=2)
scores = helper1.score_summary()
scores.sort_values(by="mean_score_accuracy", ascending=False).head(10)

Visualitzem el comportament dels hiperparàmetres.


In [ ]:
helper1.search_plot(scores.query("estimator == 'RandomForestClassifier'"), "mean_score_accuracy", "n_estimators", "max_depth")

Entrenem el model amb els hiperparàmetres escollits.


In [ ]:
rfc = RandomForestClassifier(n_estimators=32, max_depth=8)
rfc.fit(X_train[ce_cols], y_train)
rfc.score(X_test[ce_cols], y_test)

### Models utilitzant contingut i tecnologies

En aquest cas s'estudia únicament com es comporten les columnes "contingut" i "tecnologies".


In [ ]:
ct_cols = contingut_cols + tecnologies_chi2.query("p_value < 0.05").cols.values.tolist()

#### SVM

Apliquem 5-fold cross-validation sobre el conjunt d'entrenament i visualitzem quins presenten el millor comportament. S'ordenen els resultats basats en la precisió mitjana obtinguda.


In [ ]:
helper1 = EstimatorSelectionHelper(models_svm, params_svm, scoring=["accuracy"])
helper1.fit(X_train[ct_cols], y_train, n_jobs=2)

scores = helper1.score_summary()
scores.sort_values(by="mean_score_accuracy", ascending=False).head(10)

Visualitzem el comportament dels hiperparàmetres.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,5))
ax1.set(title="SVM amb kernel rbf")
helper1.search_plot(scores.query("estimator == 'SVC' and kernel == 'rbf'"), "mean_score_accuracy", "C", "gamma", ax=ax1)
ax1.set(title="SVM amb kernel linear")
helper1.search_plot(scores.query("estimator == 'SVC' and kernel == 'linear'"), "mean_score_accuracy", "C", "gamma", ax=ax2)
ax1.set(title="SVM amb kernel sigmoid")
helper1.search_plot(scores.query("estimator == 'SVC' and kernel == 'sigmoid'"), "mean_score_accuracy", "C", "gamma", ax=ax3)

Entrenem el model amb els hiperparàmetres escollits.


In [ ]:
svm = SVC(C=1, gamma=0.1, kernel='rbf')
svm.fit(X_train[ct_cols], y_train)
svm.score(X_test[ct_cols], y_test)

#### Random Forest

Apliquem 5-fold cross-validation sobre el conjunt d'entrenament i visualitzem quins presenten el millor comportament. S'ordenen els resultats basats en la precisió mitjana obtinguda.

In [ ]:
helper1 = EstimatorSelectionHelper(models_rfc, params_rfc, scoring=["accuracy"])
helper1.fit(X_train[ct_cols], y_train, n_jobs=2)
scores = helper1.score_summary()
scores.sort_values(by="mean_score_accuracy", ascending=False).head(10)

Visualitzem el comportament dels hiperparàmetres.


In [ ]:
helper1.search_plot(scores.query("estimator == 'RandomForestClassifier'"), "mean_score_accuracy", "n_estimators", "max_depth")

Entrenem el model amb els hiperparàmetres escollits.


In [ ]:
rfc = RandomForestClassifier(n_estimators=100, max_depth=2)
rfc.fit(X_train[ct_cols], y_train)
rfc.score(X_test[ct_cols], y_test)

### Models utilitzant contingut, estructura i tecnologies

En aquest cas s'estudia amb el conjunt de les columnes estudiades fins ara.


In [ ]:
cet_cols = contingut_cols + tecnologies_chi2.query("p_value < 0.05").cols.values.tolist() + ["mida", "paraules", "images"]

#### SVM

Apliquem 5-fold cross-validation sobre el conjunt d'entrenament i visualitzem quins presenten el millor comportament. S'ordenen els resultats basats en la precisió mitjana obtinguda.

In [ ]:
helper1 = EstimatorSelectionHelper(models_svm, params_svm, scoring=["accuracy"])
helper1.fit(X_train[cet_cols], y_train, n_jobs=2)

scores = helper1.score_summary()
scores.sort_values(by="mean_score_accuracy", ascending=False).head(10)

Visualitzem el comportament dels hiperparàmetres.


In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,5))
ax1.set(title="SVM amb kernel rbf")
helper1.search_plot(scores.query("estimator == 'SVC' and kernel == 'rbf'"), "mean_score_accuracy", "C", "gamma", ax=ax1)
ax1.set(title="SVM amb kernel linear")
helper1.search_plot(scores.query("estimator == 'SVC' and kernel == 'linear'"), "mean_score_accuracy", "C", "gamma", ax=ax2)
ax1.set(title="SVM amb kernel sigmoid")
helper1.search_plot(scores.query("estimator == 'SVC' and kernel == 'sigmoid'"), "mean_score_accuracy", "C", "gamma", ax=ax3)

Entrenem el model amb els hiperparàmetres escollits.


In [ ]:
svm = SVC(C=50, gamma=0.001, kernel='sigmoid')
svm.fit(X_train[cet_cols], y_train)
svm.score(X_test[cet_cols], y_test)

#### Random Forest

Apliquem 5-fold cross-validation sobre el conjunt d'entrenament i visualitzem quins presenten el millor comportament. S'ordenen els resultats basats en la precisió mitjana obtinguda.

In [ ]:
helper1 = EstimatorSelectionHelper(models_rfc, params_rfc, scoring=["accuracy", "f1"])
helper1.fit(X_train_cet, y_train_cet, n_jobs=2)
scores = helper1.score_summary()
scores.sort_values(by="mean_score_accuracy", ascending=False).head(10)

Visualitzem el comportament dels hiperparàmetres.


In [ ]:
helper1.search_plot(scores.query("estimator == 'RandomForestClassifier'"), "mean_score_accuracy", "n_estimators", "max_depth")

Entrenem el model amb els hiperparàmetres escollits.


In [ ]:
rfc = RandomForestClassifier(n_estimators=100, max_depth=8)
rfc.fit(X_train_cet, y_train_cet)
rfc.score(X_test_cet, y_test_cet)

### Conclusions models<a id='conclusions_model'></a>

Després d'haver vist com es comporten els models segons si a més de les columnes de "contingut" hi afegim l'estructura de la pàgina web o les tecnologies que aquesta utilitza podem determinar que aquestes variables extra no afecten de manera significativa al model. És més en la majoria dels casos les observacions classificades de manera incorrecta són sempre les mateixes:
* Falsos negatius: pàgines web d'empreses però que tenen poca informació i amb els paràmetres extrets de la pàgina web no es poden diferenciar de les que no són empresa.
* Falsos positius: pàgines web que no són d'empreses pero hi apareixen telèfons i correus electrònics. Recordem que aquestes dues variables juntes fan pujar entorn al 80% la probabilitat de ser empresa segons el model.

Està clar doncs, que amb les dades extretes fins ara dels dominis no és possible obtenir un model superior. El problema rau doncs, com ja s'ha explicat, en pàgines web de no empreses que contenen la informació que nosaltres a priori haviem decidit rellevant per determinar que eren una empresa. Arribata a aquest punt podem:
* Acceptar el model creat amb les dades actuals.
* Obtenir una mostra més gran de dades. Encara que una mostra major pugui millorar lleugerament el model seguim tenint el problema dels falsos positius perquè les dades són les mateixes.
* Afegir noves dades al model:
   * Anàlisi NLP.
   * Obtenir mètriques de la pàgina web utilitzant [Lighthouse](https://github.com/GoogleChrome/lighthouse). Aquest framework open-source de Google realitza anàlisi de SEO, Accessibilitat o rendiment entre d'altres a les pàgines web. Pot ser un bon enfocament per incrementar les dades encara que pot passar el mateix que amb les tecnologies utilitzades per una pàgina web: que resultin insignificants. De totes maneres cal tenir en compte que Lightouse és una proposta ja realitzada per extreure més indicadors del REPUBLIQUEM.CAT, i que per tant si s'acaba implementant s'hauran d'extreure igualment aquestes dades.

De moment, s'agafa el model SVM creat inicialment amb kernel `sigmoid`, `C=50` i `gamma=0.001` i el procedim a aplicar al conjunt total dels dominis i visualitzar-ne els resultats.

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(5,10))
probabilitat = svm_c.predict_proba(amb_contingut[contingut_cols])[:, 1]
classificacio = svm_c.predict(amb_contingut[contingut_cols])

final = pd.DataFrame({"probabilitat": probabilitat, "classificacio": classificacio})

final.groupby("classificacio").size().plot(kind='bar', figsize=(15,7), title="Classificació final", ax=ax2)
add_value_labels(ax2)
sns.distplot(final.probabilitat, bins=20, ax=ax1, kde=False)
ax1.axvline(0.5, color="red", linestyle="--")
ax1.set(title="Disitrubció probabilitat de ser empresa")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(5,10))
probabilitat = svm_c.predict_proba(amb_contingut[contingut_cols])[:, 1]
classificacio = svm_c.predict(amb_contingut[contingut_cols])

final = pd.DataFrame({"probabilitat": probabilitat, "classificacio": classificacio})

final.groupby("classificacio").size().plot(kind='bar', figsize=(15,7), title="Classificació final", ax=ax2)
add_value_labels(ax2)
sns.distplot(final.probabilitat, bins=20, ax=ax1, kde=False)
ax1.axvline(0.5, color="red", linestyle="--")
ax1.set(title="Disitrubció probabilitat de ser empresa")

### Reducció de dimensionalitat

Abans d'acabar la classificació dels dominis es realitzarà un últim anàlisi a partir de la reducció de dimensionalitat utilitzant PCA i un enfocament no supervisat amb Kmeans.

Primer de tot apliquem PCA al conjunt d'entrenament.

In [ ]:
fig3 = plt.figure(constrained_layout=True, figsize=(12, 9))
gs = fig3.add_gridspec(3, 3)
ax1 = fig3.add_subplot(gs[0:2, 0:2], projection='3d')
ax2 = fig3.add_subplot(gs[0:2, 2])
ax3 = fig3.add_subplot(gs[2, 0])
ax4 = fig3.add_subplot(gs[2, 1])
ax5 = fig3.add_subplot(gs[2, 2])

pca_c = PCAw()
dominis_pca_c = pca_c.fit_transform(X_train[contingut_cols])
ax1.scatter(xs=dominis_pca_c[:, 0], ys=dominis_pca_c[:, 1], zs=dominis_pca_c[:, 2], c=y_train)
pca_c.variance_plot(ax=ax2)
pca_c.plot_contribution(0, columns=contingut_cols, ax=ax3)
pca_c.plot_contribution(1, columns=contingut_cols, ax=ax4)
pca_c.plot_contribution(2, columns=contingut_cols, ax=ax5)

Addicionalment creem una visualització interactiva en 3D afegint els *eigenvectors*.

In [ ]:
scale = 2
no_empresa = go.Scatter3d(
    x=dominis_pca_c[y_train == 0,0],
    y = dominis_pca_c[y_train == 0,1],
    z = dominis_pca_c[y_train == 0,2],
    mode='markers',
    name="No empresa",
    hoverinfo='none',
    marker=dict(
        size=4,      
        opacity=1
    )
)

empresa = go.Scatter3d(
    x=dominis_pca_c[y_train == 1,0],
    y = dominis_pca_c[y_train == 1,1],
    z = dominis_pca_c[y_train == 1,2],
    mode='markers',
    name="No empresa",
    hoverinfo='none',
    marker=dict(
        size=4,             
        opacity=1
    )
)

arrows = []
for i, col in enumerate(contingut_cols):
    dc_1 = go.Scatter3d( x = [0,pca_c.components_.T[i][0] * scale],
                     y = [0,pca_c.components_.T[i][1] * scale],
                     z = [0,pca_c.components_.T[i][2] * scale],
                     marker = dict( size = 3,
                                    color = "rgb(84,48,5)"),
                     line = dict(color='green',
                                width = 6),
                     name = col
                     )
    arrows.append(dc_1)

data = [no_empresa, empresa] + arrows
layout = go.Layout(
)
fig = go.Figure(data=data, layout=layout)
fig.show()

Després d'aplicar PCA i visualitzar els tres primers components principals podem veure, com ja haviem determinat amb els altres models, que aquells dominis que realment són empresa són molt fàcils de classificar. A més aquells que no ho són també s'agrupen, encara que no tant clarament com els que si.
És fàcil veure com els eigenvectors formen tres grups diferents:
* Un per les xarxes socials: facebook, twitter, instagra, youtube i linkedin.
* El grup amb més influència: phone, telèfon i enllaç a contacte. Aquest fet és un clar indicador de que molts telèfons de contacts i correus electòrnics es troben en la pestanya de contacte.
* La resta de les dades com nif, enllaços avis legal, privacitat i cookies o els mots registre mercantil i domicili fiscal.

A partir d'aquesta visualització procedim a aplicar l'algorisme K-means. Primer de tot decidim el millor valor de K mitjnçant dos mètodes: la silueta i el mètode del colze. Utilitzarem fins el component principal 11 on tenim el 95% de la variància acumulada.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,5))
ax1.set(title="Silhouette method")
plot_silhouette_method(dominis_pca_c[:, 0:11], ax=ax1)
ax2.set(title="Elbow method")
plot_elbow_method(dominis_pca_c[:, 0:11], ax=ax2)


Ambdos mètodes coincideixen en que el millor valor de k és 2.

In [ ]:
kmeans_pca_c = cluster.KMeans(n_clusters=2)
kmeans_pca_c.fit(dominis_pca_c[:, 0:3])
clusters = kmeans_pca_c.labels_
c1 = kmeans_pca_c.cluster_centers_[:, 0]
c2 = kmeans_pca_c.cluster_centers_[:, 1]
c3 = kmeans_pca_c.cluster_centers_[:, 2]

fig, ax = plt.subplots()
ax = Axes3D(fig)
ax.scatter(xs=dominis_pca_c[:, 0], ys=dominis_pca_c[:, 1], zs=dominis_pca_c[:, 2], c=clusters)




# plot the centroids
ax.scatter(
    c1, c2, c3,
    s=150, marker='+',
    c='black',
    label='centroids'
)
ax.legend()

Encara que em aplicat un mètode no supervisat, disposem del valors de classificació i podem estudiar si la classificació realitzada s'apropa a la que nosaltres desitjem.

In [ ]:
predicted = kmeans_pca_c.predict(X_test_pca_c[:, 0:3])
ax = sns.heatmap(confusion_matrix(y_test, predicted), annot=True)
ax.set(xlabel="Predicted label", ylabel="True label")
print(accuracy_score(predicted, y_test))

Podem observar com la classificació del model no és ideal encara que aconsegueix un *true positive rate* (sensibilitat) de gairebé perfecte.

### Conclusions models<a id='conclusions_model'></a>

Després d'haver vist com es comporten els models segons si a més de les columnes de "contingut" hi afegim l'estructura de la pàgina web o les tecnologies que aquesta utilitza podem determinar que aquestes variables extra no afecten de manera significativa al model. És més en la majoria dels casos les observacions classificades de manera incorrecta són sempre les mateixes:
* Falsos negatius: pàgines web d'empreses però que tenen poca informació i amb els paràmetres extrets de la pàgina web no es poden diferenciar de les que no són empresa.
* Falsos positius: pàgines web que no són d'empreses pero hi apareixen telèfons i correus electrònics. Recordem que aquestes dues variables juntes fan pujar entorn al 80% la probabilitat de ser empresa segons el model.

Està clar doncs, que amb les dades extretes fins ara dels dominis no és possible obtenir un model superior. El problema rau doncs, com ja s'ha explicat, en pàgines web de no empreses que contenen la informació que nosaltres a priori haviem decidit rellevant per determinar que eren una empresa. Arribata a aquest punt podem:
* Acceptar el model creat amb les dades actuals.
* Obtenir una mostra més gran de dades. Encara que una mostra major pugui millorar lleugerament el model seguim tenint el problema dels falsos positius perquè les dades són les mateixes.
* Afegir noves dades al model:
   * Anàlisi NLP.
   * Obtenir mètriques de la pàgina web utilitzant [Lighthouse](https://github.com/GoogleChrome/lighthouse). Aquest framework open-source de Google realitza anàlisi de SEO, Accessibilitat o rendiment entre d'altres a les pàgines web. Pot ser un bon enfocament per incrementar les dades encara que pot passar el mateix que amb les tecnologies utilitzades per una pàgina web: que resultin insignificants. De totes maneres cal tenir en compte que Lightouse és una proposta ja realitzada per extreure més indicadors del REPUBLIQUEM.CAT, i que per tant si s'acaba implementant s'hauran d'extreure igualment aquestes dades.

De moment, s'agafa el model SVM creat inicialment amb kernel `sigmoid`, `C=50` i `gamma=0.001` i el procedim a aplicar al conjunt total dels dominis i visualitzar-ne els resultats.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(5,10))
probabilitat = svm_c.predict_proba(amb_contingut[contingut_cols])[:, 1]
classificacio = svm_c.predict(amb_contingut[contingut_cols])

final = pd.DataFrame({"probabilitat": probabilitat, "classificacio": classificacio})

final.groupby("classificacio").size().plot(kind='bar', figsize=(15,7), title="Classificació final", ax=ax2)
add_value_labels(ax2)
sns.distplot(final.probabilitat, bins=20, ax=ax1, kde=False)
ax1.axvline(0.5, color="red", linestyle="--")
ax1.set(title="Disitrubció probabilitat de ser empresa")